<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
        <img src="http://upload.wikimedia.org/wikipedia/en/0/0c/Mu_Sigma_Logo.jpg" style="height:75px; margin-left:0px" />
    </div>
    <div style="width: 59%; display: inline-block">
        <h1  style="text-align: center"> Social Media Analytics - Data Preparation </h1><br>
        <div style="width: 100%; text-align: center; display: inline-block;"><i>Author:</i> <strong>Varun Behl</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="2018-02-14" pubdate>13th Feb 2018</time>
            <i>Modified: </i>
            <time datetime="2018-02-21" pubdate>21st Feb 2018</time>
        </div>
    </div>
</div>

# Objective 

In this notebook we perform initial data operations on the scraped data and make it ready for training a classifier. The list of data preparation steps are as follows:

* Missing data removal
* Converting date mentions to standard format
* Drug name and user name corrections
* Filter posts with atleast one drug mention
* Vectorizing the scraped forum names
* Generating unique User_id, post_id
* Sentence tokenize all posts

In [1]:
#Importing packages
import os
import pandas as pd
import numpy as np
import re
import csv

# Date conversion
from dateutil import parser

In [2]:
# Checking current working directory 
os.getcwd()

'/home/iot/DWoW Diamonds/BMS/code_review'

In [3]:
#Reading post-level raw data 
data_post  = pd.read_csv("Final_collated_data_subset.csv")
data_post.head()

Forum                    Title        User  \
0  Cancer_Compass      Going off Anti-PD-1       lsv12   
1  Cancer_Compass  RE: Going off Anti-PD-1    Chucksan   
2  Cancer_Compass  RE: Going off Anti-PD-1     jack123   
3  Cancer_Compass         Treatment or Not  sickntired   
4  Cancer_Compass     RE: Treatment or Not      RevKev   

                        Date  \
0  Fri Oct 20, 2017 02:12 AM   
1  Fri Dec 29, 2017 09:48 AM   
2  Sat Jan 06, 2018 07:00 PM   
3  Sun Dec 03, 2017 10:39 PM   
4  Mon Dec 04, 2017 12:42 AM   

                                                Post  
0  Has anyone before being dx as NED come off the...  
1  IMO and experience do not go off the anti PD-1...  
2  Anti-PD-1 is new study..?�I actually just foun...  
3  Has anyone ever asked themselves this question...  
4  Hi Sickntired,\r\nMy point of view: Any illnes...

### STEP 1: Missing data removal

In [4]:
# Removing rows with All NA's
data_noNA = data_post.dropna(axis=0,how='any')

# Number of rows in dataset
len(data_noNA)

155

### STEP 2a: Drug name corrections

The drug name corrections are made by using a dictionary with correct drug name as values and incorrect name as keys.

In [5]:
# Sample of drug list
drug_csv = pd.read_csv('Melanoma_updated_drug_list_v2.csv')
drug_csv.head()

drug_nm corrected_drug_nm
0  Aldesleukin       Aldesleukin
1         IL 2       Aldesleukin
2          IL2       Aldesleukin
3         IL-2       Aldesleukin
4  Interleukin       Aldesleukin

In [6]:
#Subsituting exsisting drug names with the corrected drug names 

with open('Melanoma_updated_drug_list_v2.csv') as f:
    d = dict(filter(None, csv.reader(f)))

df1=pd.DataFrame(columns=['Posts_d'])
drug_terms = []
drug_terms = drug_csv['drug_nm'].values

count = 0 

try:
    assert(type(data_noNA['Post'][4])==str)
    for item1 in data_noNA['Post']:
        for item2 in drug_terms:
            try:
                item1= re.sub(r"\b(?=\w)"+re.escape(str(item2))+r"\b(?!\w)", d[item2], item1,flags=re.IGNORECASE) 
            except Exception:
                item1=item1
        count += 1
        if (count % 100)==0:
            print count
        df1=df1.append({'Posts_d':item1}, ignore_index=True)

    print "Here"
except AssertionError:
    print "The datatype of text in Post column should be string for the operations"

100
Here


In [7]:
print len(data_noNA)
print len(df1)

155
155


In [8]:
# Reset index as few rows may be deleted (Ensuring continous index)
data_Corrected = data_noNA.reset_index(drop= True)

# Appending a new column which has posts with correct drug names
data_Corrected['Posts_d'] = df1['Posts_d']

# Dropping the post columnn with faulty drug names
data_Corrected = data_Corrected.drop(['Post'],1)

In [9]:
# Saving checkpoints
data_Corrected.to_csv('Data_afterDrugCorrection.csv')

### STEP 2b: User name correction

Here we are removing ('-' and ' ') from the user name.

In [10]:
# Correcting user names 
data_Corrected['User']=data_Corrected['User'].map(lambda x:re.sub(' -','',x))


### STEP 3: Filter posts with atleast one drug mention

This step is performed because we are extracting line of therapy and posts without any drug mention will not help in any further processes. 

In [11]:
# Uploading corrected drug list file
with open("updated_drugs.txt","r") as f:
    drug = f.read()
print(drug)
regxp = drug
reg1 = re.compile(regxp,re.IGNORECASE)

Avelumab|Aldesleukin|Azacitidine|Bcg|Bendamustine Hydrochloride|Bevacizumab|Bicalutamide|Bortezomib|Cabazitaxel|Cabozantinib Malate|Capecitabine|Carboplatin|Carfilzomib|Carmustine|Cetuximab|Chlorambucil|Cisplatin|Cisplatin-Vinblastine-Dacarbazine|Cladribine|Cobimetinib|Crizotinib|Dabrafenib|Dacarbazine|Dasatinib|Decitabine|Docetaxel|Doxorubicin|Doxorubicin Pegylated Liposomal|Enzalutamide|Everolimus|Exemestane|Fludarabine Phosphate|Fluorouracil|Fulvestrant|Gemcitabine|Hydroxyurea|Ibrutinib|Idelalisib|Ifosfamide|Imatinib|Interferon|Interferon Alfa-2A|Interferon Alfa-2B|Ipilimumab|Irinotecan|Lenalidomide|Lomustine|Melphalan|Mercaptopurine|Methotrexate Sodium|Nivolumab|Obinutuzumab|Ofatumumab|Olaparib|Olaratumab|Oxaliplatin|Paclitaxel|Palbociclib|Panitumumab|Peginterferon|Peginterferon Alfa-2A|Peginterferon Alfa-2B|Pembrolizumab|Pertuzumab|Ramucirumab|Rigvir|Sunitinib Malate|T-Vec|Tamoxifen|Tamoxifen Citrate|Temozolomide|Temsirolimus|Thalidomide|Tipiracil|Trametinib|Trifluridine|Vemurafen

In [12]:
# Creating a new column with count of drug mentions in a post
data_Corrected['no_of_drugs'] = data_Corrected['Posts_d'].map(lambda x:len(reg1.findall(str(x))))

# Removing rows with no drug mentions
Data_filtered = data_Corrected[data_Corrected['no_of_drugs'] != 0]

In [13]:
# Resetting the index as few rows will be deleted upon filtering
Data_filtered = Data_filtered.reset_index(drop=True)
Data_filtered.head()

Forum                                  Title           User  \
0  Cancer_Compass                RE: Going off Anti-PD-1       Chucksan   
1  Cancer_Compass                   RE: Treatment or Not       Chucksan   
2  Cancer_Compass  RE: Stage 4 melanoma, no primary site  ilikepralinen   
3  Cancer_Compass  RE: Stage 4 melanoma, no primary site   DawnSpiering   
4  Cancer_Compass                   RE: Stage 4 Melanoma          Joan7   

                        Date  \
0  Fri Dec 29, 2017 09:48 AM   
1  Fri Dec 29, 2017 09:22 AM   
2  Mon Jun 06, 2016 08:25 PM   
3  Wed Oct 18, 2017 08:28 AM   
4  Sun Sep 03, 2017 12:33 AM   

                                             Posts_d  no_of_drugs  
0  IMO and experience do not go off the anti PD-1...            6  
1  If you have a death wish, forget treatment.\r\...            2  
2  Hello Lennonje,\r\neven i had a Metastase in m...            2  
3  On Apr 18, 2016 1:16 PM lennonje wrote: Hi,\r\...            4  
4  I was diagnosed with Stage 3b melanoma in the ...            1

In [14]:
# Number of rows in filtered dataset
print "No of rows after filtering:",len(Data_filtered)
print "Column names:", Data_filtered.columns

No of rows after filtering: 10
Column names: Index([u'Forum', u'Title', u'User', u'Date', u'Posts_d', u'no_of_drugs'], dtype='object')


### STEP 3: Converting date mentions to standard format

In [15]:
#Converting the dates in standard date-time format
Data_filtered['date'] = Data_filtered['Date'].map(lambda x:parser.parse(x))

# dropping the older date format 
Data_filtered = Data_filtered.drop(['Date'],1)

assert(type(Data_filtered['Posts_d'][1])==str)

### STEP 4: Vectorizing the scraped forum names

In this section we tag the forum names with a numeric value

In [16]:
# Counting number of posts from different forums
print Data_filtered['Forum'].value_counts()
print sum(Data_filtered['Forum'].value_counts())

Cancer_Compass             8
Melanoma_org               1
cancer_survivor_network    1
Name: Forum, dtype: int64
10


In [17]:
# Tagging forums to integer value
import warnings
warnings.filterwarnings('ignore')

Data_filtered['forum'] = 0
for i in range(0,len(Data_filtered['Forum'])) :
    if (i % 100) == 0 :
        print i
    if Data_filtered['Forum'][i]=='Melanoma_org':
        Data_filtered['forum'][i]= 1
    elif Data_filtered['Forum'][i]=='Melanoma_international':
        Data_filtered['forum'][i]= 2
    elif Data_filtered['Forum'][i]=='Cancer_Compass':
        Data_filtered['forum'][i]= 3
    else:
        Data_filtered['forum'][i]= 4

0


In [18]:
Data_filtered.head(5)

Forum                                  Title           User  \
0  Cancer_Compass                RE: Going off Anti-PD-1       Chucksan   
1  Cancer_Compass                   RE: Treatment or Not       Chucksan   
2  Cancer_Compass  RE: Stage 4 melanoma, no primary site  ilikepralinen   
3  Cancer_Compass  RE: Stage 4 melanoma, no primary site   DawnSpiering   
4  Cancer_Compass                   RE: Stage 4 Melanoma          Joan7   

                                             Posts_d  no_of_drugs  \
0  IMO and experience do not go off the anti PD-1...            6   
1  If you have a death wish, forget treatment.\r\...            2   
2  Hello Lennonje,\r\neven i had a Metastase in m...            2   
3  On Apr 18, 2016 1:16 PM lennonje wrote: Hi,\r\...            4   
4  I was diagnosed with Stage 3b melanoma in the ...            1   

                 date  forum  
0 2017-12-29 09:48:00      3  
1 2017-12-29 09:22:00      3  
2 2016-06-06 20:25:00      3  
3 2017-10-18 08:28:00      3  
4 2017-09-03 00:33:00      3

### STEP 5: Generating Unique User ID

Here we are merging with a csv where every user is already given a user id during the process of manual tagging, this is to ensure that the same id is maintained during automated process also.

In [19]:
#Uploading the dataset with user-ids already generated using manual tagging
uni_user = pd.read_csv('Unique_user.csv')
uni_user.head()

User  forum  User_id
0     Eileensulliv      1        1
1           BrianP      1        2
2          slh4448      1        3
3  FormerCaregiver      1        4
4            rylee      3        5

In [20]:
# Merging dataset for giving a User-id to every unique user
Data_User_ids = pd.merge(Data_filtered,uni_user[['User_id','User','forum']],left_on=['User','forum'],right_on=['User','forum'],how='inner')

# Ensuring there is no change in length of Dataset on merging
assert(len(Data_filtered==len(Data_User_ids)))

In [21]:
# Reset index
Data_User_ids = Data_User_ids.reset_index(drop=False)
# Removing unnecessary columns from the dataset
Data_User_ids = Data_User_ids.drop(['Title','no_of_drugs','Forum'],1)

Data_User_ids = Data_User_ids.sort_values(['User','User_id','date'],0)
# Rename the columns

Data_User_ids.columns = ['Post_id', 'User', 'Posts', 'date', 'forum', 'User_id']

In [22]:
Data_User_ids.head()

Post_id          User                                              Posts  \
1        1      Chucksan  If you have a death wish, forget treatment.\r\...   
0        0      Chucksan  IMO and experience do not go off the anti PD-1...   
3        3  DawnSpiering  On Apr 18, 2016 1:16 PM lennonje wrote: Hi,\r\...   
8        8  Dawn_Leppert  Hello, I am a Melanoma Cancer Survivor.?� I ha...   
5        5    Emptyhands  I'm so happy to hear that you've found greater...   

                 date  forum  User_id  
1 2017-12-29 09:22:00      3     2076  
0 2017-12-29 09:48:00      3     2076  
3 2017-10-18 08:28:00      3     1495  
8 2015-01-07 16:29:00      4     2262  
5 2017-09-03 01:00:00      3     2319

In [23]:
Data_User_ids.to_csv('Data_afterInitialOps.csv')

### Sentence Tokenizing the dataset 

#### Initial data cleaning steps 

In [24]:
# Data pre-processing
def clean(text):
    cleanr = re.compile(r'<[^>]+>')
    cleantext = re.sub(cleanr, '',text)  # remove HTML tags
    text = re.sub(r'(https|http)?:\/\/(\w|\.|-|\/|\?|\=|\&|\%)*\b', '', cleantext, flags=re.MULTILINE) # Remove Url
    text = re.sub(r'[^\x00-\x7F]+',' ', text) #special Characters removal - Non Ascii
    text = re.sub("\n+","\n", text)  # extra new line
    text = re.sub(' +',' ',text).strip()  # extra whitespace
    text = re.sub(r'\.+','.',text) # replace multiple dots by one dot
    text = re.sub(r'\(|\)','',text) # remove brackets but not its contents
    return text

In [25]:
# Creating another column with post afer initial preprocessing
Data_User_ids['Post_d']= Data_User_ids['Posts'].map(lambda x:clean(x))

# Dropping column with unclean posts
Data_User_Post_level = Data_User_ids.drop(['Posts'],1)

In [26]:
# Function for sentence tokenizer 
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

# List of tokenized sentences at a post level (list of list is created)
sentence_tokenized = Data_User_Post_level['Post_d'].map(lambda x:sent_tokenize(x))

[nltk_data] Downloading package punkt to /home/iot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
# Creating sentence level data with list of sentences and corresponding post-ids
sent_doc = []
doc = []
count = 0
for x in range(0,len(sentence_tokenized)):
    for i, sent in enumerate(sentence_tokenized[x]):
        print i
        a = sent
        sent_doc.append(a)
        doc.append(count)
    count +=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
0
1
2
3
4
5
6
7
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [28]:
# Pandas dataframe created for sentence tokenized data (sentence-level )
Sentence_tokenized_data = pd.DataFrame(
    {'Post_id': doc,
     'Tokenized.Sentences':sent_doc})
Sentence_tokenized_data.head()

Post_id                                Tokenized.Sentences
0        0  IMO and experience do not go off the anti PD-1...
1        0  My wife is now been in NED remission for 14 mo...
2        0  Nivolumab like Pembrolizumab were both origina...
3        0  The last year of Nivolumab or so she went from...
4        0  A PET/CT scan before another infusion showed N...

In [29]:
# Creating Sentence level data with post_id , user_id , date and forum details for each row
Final_data_sentence_level = pd.merge(Sentence_tokenized_data,Data_User_Post_level[['Post_id', 'User', 'date', 'forum', 'User_id']],left_on='Post_id',right_on='Post_id',how='outer')
Final_data_sentence_level.head()

Post_id                                Tokenized.Sentences      User  \
0        0  IMO and experience do not go off the anti PD-1...  Chucksan   
1        0  My wife is now been in NED remission for 14 mo...  Chucksan   
2        0  Nivolumab like Pembrolizumab were both origina...  Chucksan   
3        0  The last year of Nivolumab or so she went from...  Chucksan   
4        0  A PET/CT scan before another infusion showed N...  Chucksan   

                 date  forum  User_id  
0 2017-12-29 09:48:00      3     2076  
1 2017-12-29 09:48:00      3     2076  
2 2017-12-29 09:48:00      3     2076  
3 2017-12-29 09:48:00      3     2076  
4 2017-12-29 09:48:00      3     2076

In [30]:
Final_data_sentence_level.to_csv('Final_data_sentence_level_preprocessed.csv' , index = False)